In [1]:
!pip install -U pennylane==0.34.0 tensorflow==2.14.0 numpy scipy plotly seaborn pennylane-qiskit pylatexenc silence-tensorflow
# pip install pennylane qiskit qiskit_machine_learning imblearn plotly seaborn pennylane-qiskit pylatexenc silence-tensorflow

from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")
from silence_tensorflow import silence_tensorflow

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score, pairwise_distances, silhouette_samples, silhouette_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.manifold import TSNE

# classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections


# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

from time import time
from datetime import datetime
import matplotlib.cm as cm
from scipy.spatial.distance import cdist

from datetime import datetime
from joblib import Parallel, delayed

import seaborn as sns
import plotly.express as px
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import datasets
import seaborn as sns
import jax;

jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

import optax  # optimization using jax

import pennylane as qml
import pennylane.numpy as pnp

sns.set()

seed = 0
rng = np.random.default_rng(seed=seed)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 870.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of pennylane-qiskit to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of pennylane-lightning to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 29.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 47.6 MB/s eta 0:00:0000:01

In [2]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [3]:
import pennylane as qml
import qiskit
print(qml.__version__)
print(qiskit.__version__)
import pennylane_qiskit
print(pennylane_qiskit.__version__)
import tensorflow as tf
print(tf.__version__)


# Check if GPU is available
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT available")

# Check which GPU TensorFlow is using
print("TensorFlow is using", tf.test.gpu_device_name())

0.34.0
1.1.2
0.36.0


2024-08-13 18:17:45.397335: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 18:17:45.397418: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 18:17:45.397475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.14.0
GPU is NOT available
TensorFlow is using 


In [4]:
# prompt: import labels.csv and features.csv and concat them into df

import pandas as pd
# labels_df = pd.read_csv('/content/labels.csv')
# features_df = pd.read_csv('/content/features.csv')

# df = pd.concat([features_df, labels_df], axis=1)
df = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')
df.head()


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [5]:
df.shape

(918, 12)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [7]:
# Columns to encode
columns_to_encode = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

# Initialize label encoder
from sklearn.preprocessing import LabelEncoder
label_encoders = {}

# Apply label encoding to each column
for column in columns_to_encode:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le  # Save the encoder to inverse transform if needed later


In [8]:
df.nunique()

Age                50
Sex                 2
ChestPainType       4
RestingBP          67
Cholesterol       222
FastingBS           2
RestingECG          3
MaxHR             119
ExerciseAngina      2
Oldpeak            53
ST_Slope            3
HeartDisease        2
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame containing the dataset

# Define X (features) and y (target variable)
X = df.drop(columns=['HeartDisease'])  # Drop the target column
y = df['HeartDisease']  # Target column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


silence_tensorflow()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import pennylane as qml
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

trainX = X_train
testX = X_test
trainy = y_train
testy = y_test

# trainy = tf.one_hot(trainy, depth=2)
# testy = tf.one_hot(testy, depth=2)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() #create an instance of the function
trainX = scaler.fit_transform(trainX) #fit and tranform training data
testX = scaler.transform(testX) #only tranform test data

trainX_reshaped = X_train.to_numpy().reshape(X_train.shape[0], X_train.shape[1], 1)
testX_reshaped = X_test.to_numpy().reshape(X_test.shape[0], X_test.shape[1], 1)
print(trainX_reshaped.shape, testX_reshaped.shape)

X_train shape: (642, 11)
X_test shape: (276, 11)
y_train shape: (642,)
y_test shape: (276,)
(642, 11, 1) (276, 11, 1)


# Proposed

In [10]:
# prompt: git clone
!pip install pennylane silence_tensorflow keras-tuner
!git clone https://github.com/ZPZhou-lab/tfkan.git
!cd tfkan && pip install .

Cloning into 'tfkan'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 81 (delta 25), reused 71 (delta 18), pack-reused 0 (from 0)
Unpacking objects: 100% (81/81), 234.16 KiB | 2.32 MiB/s, done.
Processing /kaggle/working/tfkan
  Preparing metadata (setup.py) ... done
  Created wheel for tfkan: filename=tfkan-0.1.0-py3-none-any.whl size=12366 sha256=6f487e7ad32feb6e62aa1434ac4cc573f66bbf97ee963d2be41ff8ffc5e426f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-5dji2iss/wheels/8b/9b/ad/a4e2ec558b8612078d6d55fc115466424a4491dd7a671448d3
Successfully built tfkan


In [45]:
from tfkan import layers
from tfkan.layers import DenseKAN, Conv2DKAN, Conv1DKAN
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras import models, layers, optimizers, metrics
import pennylane as qml
import tensorflow as tf
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
tf.keras.backend.set_floatx('float64')
tf.keras.backend.clear_session()

### essentials ###
n_qubits = 4
layers = 1
data_dimension = 1  #len(YCOLS) #Y.shape[1]  ## output dimenstion according to one-hot encoding depth
dev = qml.device("default.qubit", wires=n_qubits)

## extras ###
n_block_wires = 2
n_params_block = 2
n_blocks = qml.TTN.get_n_blocks(range(n_qubits), n_block_wires)

@qml.qnode(dev)
def qnode(weights, inputs=None):
    qml.templates.AmplitudeEmbedding(features=inputs, wires=range(n_qubits),normalize=True) # first turn it true, then false, again true. It'll work
    #qml.templates.AngleEmbedding(features=inputs, wires=range(n_qubits), rotation='Z')
    # for w in weights:
    #     qml.TTN(range(n_qubits), n_block_wires, block, n_params_block, w)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (layers,n_qubits,3)}

qlayer_1 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer1')
qlayer_2 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer2')
qlayer_3 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer3')

In [12]:
# weights_init = 2*np.pi * np.random.randn(num_layers,n_blocks, n_params_block, requires_grad=True)
# bias_init = np.array(0.0, requires_grad=True)
# print(weights_init, bias_init)
# circuit(weights_init,np.asarray(X_train.iloc[0]))
# dev._circuit.draw('mpl', plot_barriers=True, reverse_bits=True)

## Model-1: BiLSTMKANnet

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

batch_size = 10
# Define the input layer
inputs = layers.Input(shape=(trainX_reshaped.shape[1], 1))

class MyHyperModel(HyperModel):
    def build(self, hp):
        model1 = layers.Bidirectional(layers.LSTM(
            hp.Int('units1', min_value=32, max_value=128, step=32),
            activation='relu', return_sequences=True))(inputs)

        model1 = layers.Bidirectional(layers.LSTM(
            hp.Int('units2', min_value=16, max_value=64, step=16),
            activation='relu', return_sequences=True))(model1)

        model1 = layers.Flatten()(model1)
        model1 = layers.Dropout(0.2)(model1)

        model1 = layers.Dense(hp.Int('dense_units1', min_value=128, max_value=512, step=64), activation='relu')(model1)
        model1 = layers.Dropout(0.2)(model1)

        model1 = DenseKAN(units=hp.Int('units', min_value=32, max_value=256, step=32), grid_size=3)(model1)
        model1 = layers.Dropout(0.2)(model1)

        model1 = DenseKAN(32, grid_size=3)(model1)
        model1 = layers.Dropout(0.2)(model1)

        outputs = layers.Dense(data_dimension, activation='sigmoid')(model1)

        model = models.Model(inputs=inputs, outputs=outputs)

        learning_rate = 1e-3
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        model.compile(
            loss='binary_crossentropy',
            optimizer=optimizer,
            metrics='accuracy'
        )

        return model

# Define the hypermodel
hypermodel = MyHyperModel()

# Initialize the tuner
tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='hyperband_tuning'
)

# Print the results summary
tuner.results_summary()

# Search for the best hyperparameters
tuner.search(trainX, trainy,
             batch_size=batch_size,
             epochs=50,
             validation_data=(testX, testy),
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

Trial 30 Complete [00h 00m 34s]
val_accuracy: 0.8405797101449275

Best val_accuracy So Far: 0.8695652173913043
Total elapsed time: 00h 11m 09s


In [14]:
tuner.results_summary()

Results summary
Results in my_dir/hyperband_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0026 summary
Hyperparameters:
units1: 64
units2: 64
dense_units1: 320
units: 256
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8695652173913043

Trial 0027 summary
Hyperparameters:
units1: 64
units2: 16
dense_units1: 128
units: 128
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8695652173913043

Trial 0016 summary
Hyperparameters:
units1: 32
units2: 48
dense_units1: 448
units: 192
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.8659420289855072

Trial 0013 summary
Hyperparameters:
units1: 32
units2: 48
dense_units1: 448
units: 192
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0000
Score: 0.8623188405797102

Trial 0015 summary
Hyperparameters:
units1: 32
units2: 48
dense_units1: 512
units: 64
tuner/

In [38]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Lambda, LSTM
from keras.initializers import random_uniform
batch_size = 10
# Define the input layer
inputs = layers.Input(shape=(trainX_reshaped.shape[1], 1))

model1 = layers.Bidirectional(LSTM(64, activation='relu', return_sequences=True))(inputs)
model1 = layers.Bidirectional(LSTM(64, activation='relu', return_sequences=True))(model1)
model1 = layers.Flatten()(model1)
model1 = Dropout(0.20)(model1)
model1 = Dense(320, activation='relu')(model1)
model1 = Dropout(0.20)(model1)
model1 = DenseKAN(256, grid_size=3)(model1)
model1 = Dropout(0.20)(model1)
model1 = DenseKAN(32, grid_size=3)(model1)
model1 = Dropout(0.20)(model1)
outputs = layers.Dense(data_dimension, activation='sigmoid')(model1)
model = models.Model(inputs=inputs, outputs=outputs)


class ReduceLRBacktrack(tf.keras.callbacks.ReduceLROnPlateau):
    def __init__(self, best_path, *args, **kwargs):
        super(ReduceLRBacktrack, self).__init__(*args, **kwargs)
        self.best_path = best_path

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            logging.warning('Reduce LR on plateau conditioned on metric `%s` '
                            'which is not available. Available metrics are: %s',
                             self.monitor, ','.join(list(logs.keys())))
        if not self.monitor_op(current, self.best): # not new best
            if not self.in_cooldown(): # and we're not in cooldown
                if self.wait+1 >= self.patience: # going to reduce lr
                    # load best model so far
                    # print("\n Backtracking to best model before reducting LR")
                    self.model.load_weights(self.best_path)

        super().on_epoch_end(epoch, logs) # actually reduce LR

model_path = 'bilstmkan_best.hdf5' # hdf5 for TF v2 2.14.0, for others h5 or keras or tf

lr_reducer = ReduceLRBacktrack(
    best_path = model_path,
    monitor="val_loss",
    factor=0.5,
    patience=5,
    mode='min',
    min_lr=1e-10
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path, save_weights_only=True, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1
)

# Compile the model
learning_rate = 1e-3
optimizer = optimizers.Adam(learning_rate=learning_rate)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

from tensorflow.keras.metrics import BinaryAccuracy

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics='accuracy'
)
history = model.fit(
    trainX,
    trainy,
    # batch_size=batch_size,
    epochs = 100,
    callbacks=[checkpoint, lr_reducer, early_stopping],
    validation_data = [testX, testy]
)

Epoch 1/100
20/21 [===========================>..] - ETA: 0s - loss: 0.7115 - accuracy: 0.5078
Epoch 1: val_accuracy improved from -inf to 0.59420, saving model to bilstmkan_best.hdf5
21/21 [==============================] - 10s 130ms/step - loss: 0.7114 - accuracy: 0.5078 - val_loss: 0.6776 - val_accuracy: 0.5942 - lr: 1.0000e-03
Epoch 2/100
20/21 [===========================>..] - ETA: 0s - loss: 0.7165 - accuracy: 0.4719
Epoch 2: val_accuracy did not improve from 0.59420
21/21 [==============================] - 2s 78ms/step - loss: 0.7162 - accuracy: 0.4720 - val_loss: 0.6948 - val_accuracy: 0.4058 - lr: 1.0000e-03
Epoch 3/100
20/21 [===========================>..] - ETA: 0s - loss: 0.6995 - accuracy: 0.5016
Epoch 3: val_accuracy did not improve from 0.59420
21/21 [==============================] - 2s 77ms/step - loss: 0.6995 - accuracy: 0.5016 - val_loss: 0.6585 - val_accuracy: 0.5942 - lr: 1.0000e-03
Epoch 4/100
20/21 [===========================>..] - ETA: 0s - loss: 0.6598 - acc

In [39]:
best_model = model
best_model.evaluate(testX, testy)
best_model.evaluate(trainX, trainy)
predy = best_model.predict(testX)

# train_labels_pred= (predy = model.predict(trainX) > 0.5).astype(int).flatten()
train_labels_real=trainy
rounded_labels_pred= (predy > 0.5).astype(int).flatten()
rounded_labels_real=testy

labels = ['Normal','Heart Disease']
from sklearn.metrics import classification_report, roc_auc_score
# print(classification_report(rounded_labels_real, rounded_labels_pred, target_names=labels))
print(classification_report(testy, rounded_labels_pred, target_names=labels))

from sklearn.metrics import classification_report, matthews_corrcoef, cohen_kappa_score

# Calculate MCC and Kappa scores
mcc = matthews_corrcoef(testy, rounded_labels_pred)
kappa = cohen_kappa_score(testy, rounded_labels_pred)

# Print MCC and Kappa scores
print(f"Matthews Correlation Coefficient (MCC): {mcc}")
print(f"Cohen's Kappa Score: {kappa}")
print("roc: ", roc_auc_score(rounded_labels_real, predy))

9/9 [==============================] - 1s 22ms/step
               precision    recall  f1-score   support

       Normal       0.86      0.84      0.85       112
Heart Disease       0.89      0.91      0.90       164

     accuracy                           0.88       276
    macro avg       0.88      0.87      0.88       276
 weighted avg       0.88      0.88      0.88       276

Matthews Correlation Coefficient (MCC): 0.7512039489056183
Cohen's Kappa Score: 0.7510114816839804
roc:  0.9503484320557491


# Model-2: QCKANnet

In [53]:
from tfkan.layers import DenseKAN, Conv2DKAN, Conv1DKAN
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import models, layers, optimizers, metrics
import pennylane as qml
import tensorflow as tf
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
tf.keras.backend.set_floatx('float64')
tf.keras.backend.clear_session()
from pennylane import numpy as np

### essentials ###
n_qubits = 4
layers = 4
data_dimension = 1  #len(YCOLS) #Y.shape[1]  ## output dimenstion according to one-hot encoding depth
dev = qml.device("default.qubit", wires=n_qubits)

## extras ###
n_block_wires = 2
n_params_block = 2
n_blocks = qml.TTN.get_n_blocks(range(n_qubits), n_block_wires)

@qml.qnode(dev)
def qnode(weights, inputs=None):
    qml.templates.AmplitudeEmbedding(features=inputs, wires=range(n_qubits),normalize=True, pad_with=True) # first turn it true, then false, again true. It'll work
    #qml.templates.AngleEmbedding(features=inputs, wires=range(n_qubits), rotation='Z')
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (layers, n_qubits, 3)}

qlayer_1 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer1')
qlayer_2 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer2')
qlayer_3 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer3')


In [58]:
import tensorflow as tf
from tensorflow.keras import layers, models
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

batch_size = 10
# Define the input layer
inputs = layers.Input(shape=(trainX_reshaped.shape[1], 1))

class MyHyperModel(HyperModel):
    def build(self, hp):
        model4 = Conv1DKAN(filters=hp.Int('filters_1', min_value=32, max_value=128, step=32), kernel_size=3, strides=2, padding='valid', kan_kwargs={'grid_size': 3})(inputs)
        model4 = Conv1DKAN(filters=hp.Int('filters_1', min_value=32, max_value=128, step=32), kernel_size=2, strides=2, padding='valid', kan_kwargs={'grid_size': 3})(model4)
        model4 = layers.Dropout(0.2)(model4)
        model4 = layers.Flatten()(model4)
        model4= DenseKAN((2**n_qubits)*3)(model4)
        x_1, x_2, x_3 = tf.split(model4, 3, axis=1)
        x_1 = qlayer_1(x_1)
        x_2 = qlayer_2(x_2)
        x_3 = qlayer_3(x_3)
        model4 = tf.concat([x_1, x_2, x_3], axis=1)
        model4 = DenseKAN(256, grid_size=3)(model4)
        model4 = layers.Dropout(0.2)(model4)
        model4 = DenseKAN(32, grid_size=3)(model4)
        model4 = layers.Dropout(0.2)(model4)

        outputs = layers.Dense(data_dimension, activation='sigmoid')(model4)

        model = models.Model(inputs=inputs, outputs=outputs)

        learning_rate = 1e-3
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        model.compile(
            loss='binary_crossentropy',
            optimizer=optimizer,
            metrics='accuracy'
        )

        return model

# Define the hypermodel
hypermodel = MyHyperModel()

# Initialize the tuner
tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='ht0'
)

# Print the results summary
tuner.results_summary()

# Search for the best hyperparameters
tuner.search(trainX, trainy,
             batch_size=batch_size,
             epochs=50,
             validation_data=(testX, testy),
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

Trial 4 Complete [00h 01m 53s]
val_accuracy: 0.8152173913043478

Best val_accuracy So Far: 0.8188405797101449
Total elapsed time: 00h 07m 27s


In [59]:
tuner.results_summary()

Results summary
Results in my_dir/ht0
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0002 summary
Hyperparameters:
filters_1: 96
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.8188405797101449

Trial 0003 summary
Hyperparameters:
filters_1: 128
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.8152173913043478

Trial 0000 summary
Hyperparameters:
filters_1: 64
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.8079710144927537

Trial 0001 summary
Hyperparameters:
filters_1: 32
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.8079710144927537


In [54]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Lambda, LSTM
from keras.initializers import random_uniform
batch_size = 10
# Define the input layer
inputs = layers.Input(shape=(trainX_reshaped.shape[1], 1))


model4 = Conv1DKAN(filters=96, kernel_size=3, strides=2, padding='valid', kan_kwargs={'grid_size': 3})(inputs)
model4 = Conv1DKAN(filters=96, kernel_size=2, strides=2, padding='valid', kan_kwargs={'grid_size': 3})(model4)
model4 = layers.Dropout(0.2)(model4)
model4 = layers.Flatten()(model4)
model4= DenseKAN((2**n_qubits)*3)(model4)
x_1, x_2, x_3 = tf.split(model4, 3, axis=1)
x_1 = qlayer_1(x_1)
x_2 = qlayer_2(x_2)
x_3 = qlayer_3(x_3)
model4 = tf.concat([x_1, x_2, x_3], axis=1)
model4 = DenseKAN(256, grid_size=3)(model4)
model4 = layers.Dropout(0.2)(model4)
model4 = DenseKAN(32, grid_size=3)(model4)
model4 = layers.Dropout(0.2)(model4)
outputs = layers.Dense(data_dimension, activation='sigmoid')(model4)
model = models.Model(inputs=inputs, outputs=outputs)


class ReduceLRBacktrack(tf.keras.callbacks.ReduceLROnPlateau):
    def __init__(self, best_path, *args, **kwargs):
        super(ReduceLRBacktrack, self).__init__(*args, **kwargs)
        self.best_path = best_path

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            logging.warning('Reduce LR on plateau conditioned on metric `%s` '
                            'which is not available. Available metrics are: %s',
                             self.monitor, ','.join(list(logs.keys())))
        if not self.monitor_op(current, self.best): # not new best
            if not self.in_cooldown(): # and we're not in cooldown
                if self.wait+1 >= self.patience: # going to reduce lr
                    # load best model so far
                    # print("\n Backtracking to best model before reducting LR")
                    self.model.load_weights(self.best_path)

        super().on_epoch_end(epoch, logs) # actually reduce LR

model_path = 'qcnn_4qubit_4layer.hdf5' # hdf5 for TF v2 2.14.0, for others h5 or keras or tf

lr_reducer = ReduceLRBacktrack(
    best_path = model_path,
    monitor="val_loss",
    factor=0.5,
    patience=5,
    mode='min',
    min_lr=1e-10
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path, save_weights_only=True, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1
)

# Compile the model
learning_rate = 1e-3
optimizer = optimizers.Adam(learning_rate=learning_rate)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

from tensorflow.keras.metrics import BinaryAccuracy

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics='accuracy'
)
history = model.fit(
    trainX,
    trainy,
    # batch_size=batch_size,
    epochs = 100,
    callbacks=[checkpoint, lr_reducer, early_stopping],
    validation_data = [testX, testy]
)

Epoch 1/100
21/21 [==============================] - ETA: 0s - loss: 0.6570 - accuracy: 0.6184
Epoch 1: val_accuracy improved from -inf to 0.78623, saving model to qcnn_4qubit_4layer.hdf5
21/21 [==============================] - 33s 2s/step - loss: 0.6570 - accuracy: 0.6184 - val_loss: 0.5391 - val_accuracy: 0.7862 - lr: 1.0000e-03
Epoch 2/100
21/21 [==============================] - ETA: 0s - loss: 0.5120 - accuracy: 0.7710
Epoch 2: val_accuracy did not improve from 0.78623
21/21 [==============================] - 32s 2s/step - loss: 0.5120 - accuracy: 0.7710 - val_loss: 0.5084 - val_accuracy: 0.7681 - lr: 1.0000e-03
Epoch 3/100
21/21 [==============================] - ETA: 0s - loss: 0.4852 - accuracy: 0.7928
Epoch 3: val_accuracy did not improve from 0.78623
21/21 [==============================] - 32s 2s/step - loss: 0.4852 - accuracy: 0.7928 - val_loss: 0.4997 - val_accuracy: 0.7645 - lr: 1.0000e-03
Epoch 4/100
21/21 [==============================] - ETA: 0s - loss: 0.4730 - accu

# Model 3: QDenseKANnet

In [62]:
from tfkan import layers
from tfkan.layers import DenseKAN, Conv2DKAN, Conv1DKAN
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import models, layers, optimizers, metrics
import pennylane as qml
import tensorflow as tf
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
tf.keras.backend.set_floatx('float64')
tf.keras.backend.clear_session()
from pennylane import numpy as np

### essentials ###
n_qubits = 4
layers = 1
data_dimension = 1  #len(YCOLS) #Y.shape[1]  ## output dimenstion according to one-hot encoding depth
dev = qml.device("default.qubit", wires=n_qubits)

## extras ###
n_block_wires = 2
n_params_block = 2
n_blocks = qml.TTN.get_n_blocks(range(n_qubits), n_block_wires)

@qml.qnode(dev)
def qnode(weights, inputs=None):
    qml.templates.AmplitudeEmbedding(features=inputs, wires=range(n_qubits),normalize=True, pad_with=True) # first turn it true, then false, again true. It'll work
    #qml.templates.AngleEmbedding(features=inputs, wires=range(n_qubits), rotation='Z')
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (layers, n_qubits, 3)}

qlayer_1 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer1')
qlayer_2 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer2')
qlayer_3 = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits, name='QuantumLayer3')


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

batch_size = 10
# Define the input layer
inputs = layers.Input(shape=(trainX_reshaped.shape[1], 1))

class MyHyperModel(HyperModel):
    def build(self, hp):
        model4 = DenseKAN(units=hp.Int('units_1', min_value=16, max_value=256, step=32), grid_size=3)(inputs)
        model4 = layers.Flatten()(model4)
        model4 = layers.Dropout(0.2)(model4)
        model4 = DenseKAN(units=hp.Int('units_2', min_value=16, max_value=256, step=32), grid_size=3)(model4)
        model4 = layers.Dropout(0.2)(model4)
        model4= DenseKAN((2**n_qubits)*3)(model4)
        x_1, x_2, x_3 = tf.split(model4, 3, axis=1)
        x_1 = qlayer_1(x_1)
        x_2 = qlayer_2(x_2)
        x_3 = qlayer_3(x_3)
        model4 = tf.concat([x_1, x_2, x_3], axis=1)
        model4 = DenseKAN(units=hp.Int('units_3', min_value=16, max_value=256, step=32), grid_size=3)(model4)
        model4 = layers.Dropout(0.2)(model4)
        model4 = DenseKAN(32, grid_size=3)(model4)
        model4 = layers.Dropout(0.2)(model4)

        outputs = layers.Dense(data_dimension, activation='sigmoid')(model4)

        model = models.Model(inputs=inputs, outputs=outputs)

        learning_rate = 1e-3
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        model.compile(
            loss='binary_crossentropy',
            optimizer=optimizer,
            metrics='accuracy'
        )

        return model

# Define the hypermodel
hypermodel = MyHyperModel()

# Initialize the tuner
tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='ht1'
)

# Print the results summary
tuner.results_summary()

# Search for the best hyperparameters
tuner.search(trainX, trainy,
             batch_size=batch_size,
             epochs=50,
             validation_data=(testX, testy),
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

Trial 26 Complete [00h 03m 32s]
val_accuracy: 0.8695652173913043

Best val_accuracy So Far: 0.8840579710144928
Total elapsed time: 01h 12m 44s

Search: Running Trial #27

Value             |Best Value So Far |Hyperparameter
208               |112               |units_1
16                |176               |units_2
144               |112               |units_3
10                |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
0                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/10
65/65 [==============================] - 55s 846ms/step - loss: 0.7053 - accuracy: 0.5421 - val_loss: 0.6827 - val_accuracy: 0.6957
Epoch 2/10
65/65 [==============================] - 53s 817ms/step - loss: 0.6143 - accuracy: 0.6558 - val_loss: 0.5216 - val_accuracy: 0.7754
Epoch 3/10
65/65 [==============================] - 53s 820ms/step - loss: 0.4618 - accuracy: 0.8162 - val_loss: 0.5658 - val_accuracy: 0.75

In [ ]:
tuner.results_summary()

In [36]:
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Lambda, LSTM
from keras.initializers import random_uniform
batch_size = 10
# Define the input layer
inputs = layers.Input(shape=(trainX_reshaped.shape[1], 1))


model2 = DenseKAN(112, grid_size=3)(inputs)
model2 = layers.Flatten()(model2)
model2 = layers.Dropout(0.2)(model2)
model2 = DenseKAN(176, grid_size=3)(model2)
model2 = layers.Dropout(0.2)(model2)
# model2= DenseKAN((3*n_qubits))(model2)
model2= DenseKAN((2**n_qubits)*3)(model2)
x_1, x_2, x_3 = tf.split(model2, 3, axis=1)
x_1 = qlayer_1(x_1)
x_2 = qlayer_2(x_2)
x_3 = qlayer_3(x_3)
model2 = tf.concat([x_1, x_2, x_3], axis=1)
model2 = DenseKAN(112, grid_size=3)(model2)
model2 = layers.Dropout(0.2)(model2)
model2 = DenseKAN(32, grid_size=3)(model2)
model2 = layers.Dropout(0.2)(model2)
outputs = layers.Dense(data_dimension, activation='sigmoid')(model2)
model = models.Model(inputs=inputs, outputs=outputs)


class ReduceLRBacktrack(tf.keras.callbacks.ReduceLROnPlateau):
    def __init__(self, best_path, *args, **kwargs):
        super(ReduceLRBacktrack, self).__init__(*args, **kwargs)
        self.best_path = best_path

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            logging.warning('Reduce LR on plateau conditioned on metric `%s` '
                            'which is not available. Available metrics are: %s',
                             self.monitor, ','.join(list(logs.keys())))
        if not self.monitor_op(current, self.best): # not new best
            if not self.in_cooldown(): # and we're not in cooldown
                if self.wait+1 >= self.patience: # going to reduce lr
                    # load best model so far
                    # print("\n Backtracking to best model before reducting LR")
                    self.model.load_weights(self.best_path)

        super().on_epoch_end(epoch, logs) # actually reduce LR

model_path = 'qdense_best.hdf5' # hdf5 for TF v2 2.14.0, for others h5 or keras or tf

lr_reducer = ReduceLRBacktrack(
    best_path = model_path,
    monitor="val_loss",
    factor=0.5,
    patience=5,
    mode='min',
    min_lr=1e-10
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_path, save_weights_only=True, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1
)

# Compile the model
learning_rate = 1e-3
optimizer = optimizers.Adam(learning_rate=learning_rate)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

from tensorflow.keras.metrics import BinaryAccuracy

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics='accuracy'
)
history = model.fit(
    trainX,
    trainy,
    # batch_size=batch_size,
    epochs = 100,
    callbacks=[checkpoint, lr_reducer, early_stopping],
    validation_data = [testX, testy]
)

Epoch 1/100
21/21 [==============================] - ETA: 0s - loss: 0.6840 - accuracy: 0.5670
Epoch 1: val_accuracy improved from -inf to 0.80797, saving model to qdense_best.hdf5
21/21 [==============================] - 23s 1s/step - loss: 0.6840 - accuracy: 0.5670 - val_loss: 0.6677 - val_accuracy: 0.8080 - lr: 1.0000e-03
Epoch 2/100
21/21 [==============================] - ETA: 0s - loss: 0.5755 - accuracy: 0.7632
Epoch 2: val_accuracy improved from 0.80797 to 0.81522, saving model to qdense_best.hdf5
21/21 [==============================] - 23s 1s/step - loss: 0.5755 - accuracy: 0.7632 - val_loss: 0.4130 - val_accuracy: 0.8152 - lr: 1.0000e-03
Epoch 3/100
21/21 [==============================] - ETA: 0s - loss: 0.4243 - accuracy: 0.8380
Epoch 3: val_accuracy improved from 0.81522 to 0.84058, saving model to qdense_best.hdf5
21/21 [==============================] - 23s 1s/step - loss: 0.4243 - accuracy: 0.8380 - val_loss: 0.3673 - val_accuracy: 0.8406 - lr: 1.0000e-03
Epoch 4/100
2

In [37]:
best_model = model
best_model.evaluate(testX, testy)
best_model.evaluate(trainX, trainy)
predy = best_model.predict(testX)

# train_labels_pred= (predy = model.predict(trainX) > 0.5).astype(int).flatten()
train_labels_real=trainy
rounded_labels_pred= (predy > 0.5).astype(int).flatten()
rounded_labels_real=testy

labels = ['Normal','Heart Disease']
from sklearn.metrics import classification_report, roc_auc_score
# print(classification_report(rounded_labels_real, rounded_labels_pred, target_names=labels))
print(classification_report(testy, rounded_labels_pred, target_names=labels))

from sklearn.metrics import classification_report, matthews_corrcoef, cohen_kappa_score

# Calculate MCC and Kappa scores
mcc = matthews_corrcoef(testy, rounded_labels_pred)
kappa = cohen_kappa_score(testy, rounded_labels_pred)

# Print MCC and Kappa scores
print(f"Matthews Correlation Coefficient (MCC): {mcc}")
print(f"Cohen's Kappa Score: {kappa}")
print("roc: ", roc_auc_score(rounded_labels_real, predy))

9/9 [==============================] - 3s 378ms/step
               precision    recall  f1-score   support

       Normal       0.84      0.88      0.86       112
Heart Disease       0.91      0.88      0.90       164

     accuracy                           0.88       276
    macro avg       0.87      0.88      0.88       276
 weighted avg       0.88      0.88      0.88       276

Matthews Correlation Coefficient (MCC): 0.7543361931280064
Cohen's Kappa Score: 0.7538112228349011
roc:  0.9353222996515679


**4 qubit 4 layer**

In [55]:
best_model = model
best_model.evaluate(testX, testy)
best_model.evaluate(trainX, trainy)
predy = best_model.predict(testX)

# train_labels_pred= (predy = model.predict(trainX) > 0.5).astype(int).flatten()
train_labels_real=trainy
rounded_labels_pred= (predy > 0.5).astype(int).flatten()
rounded_labels_real=testy

labels = ['Normal','Heart Disease']
from sklearn.metrics import classification_report, roc_auc_score
# print(classification_report(rounded_labels_real, rounded_labels_pred, target_names=labels))
print(classification_report(testy, rounded_labels_pred, target_names=labels))

from sklearn.metrics import classification_report, matthews_corrcoef, cohen_kappa_score

# Calculate MCC and Kappa scores
mcc = matthews_corrcoef(testy, rounded_labels_pred)
kappa = cohen_kappa_score(testy, rounded_labels_pred)

# Print MCC and Kappa scores
print(f"Matthews Correlation Coefficient (MCC): {mcc}")
print(f"Cohen's Kappa Score: {kappa}")
print("roc: ", roc_auc_score(rounded_labels_real, predy))

9/9 [==============================] - 4s 482ms/step
               precision    recall  f1-score   support

       Normal       0.77      0.84      0.80       112
Heart Disease       0.88      0.83      0.86       164

     accuracy                           0.83       276
    macro avg       0.83      0.83      0.83       276
 weighted avg       0.84      0.83      0.83       276

Matthews Correlation Coefficient (MCC): 0.6610391110057477
Cohen's Kappa Score: 0.6592226755421946
roc:  0.9075566202090593


**4 qubit 3 layer**

In [51]:
best_model = model
best_model.evaluate(testX, testy)
best_model.evaluate(trainX, trainy)
predy = best_model.predict(testX)

# train_labels_pred= (predy = model.predict(trainX) > 0.5).astype(int).flatten()
train_labels_real=trainy
rounded_labels_pred= (predy > 0.5).astype(int).flatten()
rounded_labels_real=testy

labels = ['Normal','Heart Disease']
from sklearn.metrics import classification_report, roc_auc_score
# print(classification_report(rounded_labels_real, rounded_labels_pred, target_names=labels))
print(classification_report(testy, rounded_labels_pred, target_names=labels))

from sklearn.metrics import classification_report, matthews_corrcoef, cohen_kappa_score

# Calculate MCC and Kappa scores
mcc = matthews_corrcoef(testy, rounded_labels_pred)
kappa = cohen_kappa_score(testy, rounded_labels_pred)

# Print MCC and Kappa scores
print(f"Matthews Correlation Coefficient (MCC): {mcc}")
print(f"Cohen's Kappa Score: {kappa}")
print("roc: ", roc_auc_score(rounded_labels_real, predy))

9/9 [==============================] - 4s 418ms/step
               precision    recall  f1-score   support

       Normal       0.75      0.82      0.79       112
Heart Disease       0.87      0.82      0.84       164

     accuracy                           0.82       276
    macro avg       0.81      0.82      0.81       276
 weighted avg       0.82      0.82      0.82       276

Matthews Correlation Coefficient (MCC): 0.6313246493383884
Cohen's Kappa Score: 0.6295898647197766
roc:  0.9069577526132404


**4 qubit 2 layer**

In [48]:
best_model = model
best_model.evaluate(testX, testy)
best_model.evaluate(trainX, trainy)
predy = best_model.predict(testX)

# train_labels_pred= (predy = model.predict(trainX) > 0.5).astype(int).flatten()
train_labels_real=trainy
rounded_labels_pred= (predy > 0.5).astype(int).flatten()
rounded_labels_real=testy

labels = ['Normal','Heart Disease']
from sklearn.metrics import classification_report, roc_auc_score
# print(classification_report(rounded_labels_real, rounded_labels_pred, target_names=labels))
print(classification_report(testy, rounded_labels_pred, target_names=labels))

from sklearn.metrics import classification_report, matthews_corrcoef, cohen_kappa_score

# Calculate MCC and Kappa scores
mcc = matthews_corrcoef(testy, rounded_labels_pred)
kappa = cohen_kappa_score(testy, rounded_labels_pred)

# Print MCC and Kappa scores
print(f"Matthews Correlation Coefficient (MCC): {mcc}")
print(f"Cohen's Kappa Score: {kappa}")
print("roc: ", roc_auc_score(rounded_labels_real, predy))

9/9 [==============================] - 3s 344ms/step
               precision    recall  f1-score   support

       Normal       0.76      0.86      0.80       112
Heart Disease       0.89      0.81      0.85       164

     accuracy                           0.83       276
    macro avg       0.82      0.83      0.83       276
 weighted avg       0.84      0.83      0.83       276

Matthews Correlation Coefficient (MCC): 0.658247800429101
Cohen's Kappa Score: 0.6542275295873761
roc:  0.9101154181184669


**4 qubit 1 layer**

In [42]:
best_model = model
best_model.evaluate(testX, testy)
best_model.evaluate(trainX, trainy)
predy = best_model.predict(testX)

# train_labels_pred= (predy = model.predict(trainX) > 0.5).astype(int).flatten()
train_labels_real=trainy
rounded_labels_pred= (predy > 0.5).astype(int).flatten()
rounded_labels_real=testy

labels = ['Normal','Heart Disease']
from sklearn.metrics import classification_report, roc_auc_score
# print(classification_report(rounded_labels_real, rounded_labels_pred, target_names=labels))
print(classification_report(testy, rounded_labels_pred, target_names=labels))

from sklearn.metrics import classification_report, matthews_corrcoef, cohen_kappa_score

# Calculate MCC and Kappa scores
mcc = matthews_corrcoef(testy, rounded_labels_pred)
kappa = cohen_kappa_score(testy, rounded_labels_pred)

# Print MCC and Kappa scores
print(f"Matthews Correlation Coefficient (MCC): {mcc}")
print(f"Cohen's Kappa Score: {kappa}")
print("roc: ", roc_auc_score(rounded_labels_real, predy))

9/9 [==============================] - 2s 272ms/step
               precision    recall  f1-score   support

       Normal       0.77      0.84      0.80       112
Heart Disease       0.88      0.83      0.86       164

     accuracy                           0.83       276
    macro avg       0.83      0.83      0.83       276
 weighted avg       0.84      0.83      0.83       276

Matthews Correlation Coefficient (MCC): 0.6610391110057477
Cohen's Kappa Score: 0.6592226755421946
roc:  0.9057055749128919
